In [2]:
import sources.logins as lg
from sqlite3 import connect
import sources.parts_list_0802 as pl
import pandas as pd
from datetime import datetime as dt
import xlwings as xw

In [3]:
conn = connect('quotation.db')


In [ ]:
pr110 = xw.Book('New ML/PR110_Stock_in_Aug_08082023.xlsx')


In [9]:
q = 'select * from prices'
prices = pd.read_sql(q,conn)
prices.to_excel('inspection/prices_231113.xlsx',index = False)

In [ ]:
sh = pr110.sheets[0]
part_list = []
for i in range(16,1000):
#     print(i)
    try:
        if sh.range(f'C{i}').value.startswith('FFVN'):
            info = sh.range(f'C{i}').value[6:-1]
        if sh.range(f'C{i}').value.startswith('Stock'):
            info = 'Stock'
#         print(info)
    except Exception as e:
        print (i,e)
    if sh.range(f'D{i}').value != None: 
        print(i,sh.range(f'D{i}').value.strip(),info.strip(),sh.range(f'F{i}').value)
        part_list.append(sh.range(f'D{i}').value.strip())
    if sh.range(f'D{i}').value == 'Reviewed by':
        print(f'Stop {i}')
        break
        
parts = tuple(part_list)

In [6]:
part_list = '''
113Y200780E
28A1259770C


'''
parts = tuple(part_list.strip().split('\n'))
parts 


('113Y200780E', '28A1259770C')

In [ ]:
pr110.sheets.add('Sheet1',after=sh.name)
sh1 = pr110.sheets('Sheet1')

In [7]:
# parts = ('858Y200030D','858Y200030D')
q = f'''
        SELECT [Parts Name(EN)] AS [PART_DESCRIPTION],PN.VIE,
            CASE 
                WHEN substr(P.[Article Code],4,1) IN ('Y','N','S') THEN 'F'
                WHEN substr(P.[Article Code],1,1) = 'J' THEN 'F'
                ELSE 'FW12G' END AS SAP,
            P.[Article Code] as part_num,p.[Subsidiaries (U$)] AS[Sub Price],p.[Cost of Goods sold] as [FFVN Price],
            p.[Dealer Standard Price (wiout VAT)] as [Dealer Price]



            FROM prices P
            LEFT JOIN part_name pn ON TRIM(REPLACE(p.[Parts Name(EN)],'ROHS2','')) = pn.part_description
            WHERE part_num IN {parts}
        
     '''

parts_list = pd.read_sql(q,conn)
for part in parts:
    if part not in list(parts_list['part_num']):print(part)
parts_list

113Y200780E


,PART_DESCRIPTION,Vie,SAP,part_num,Sub Price,FFVN Price,Dealer Price
0,IRIS ASSY 2 ROHS2,None,FW12G,28A1259770C,558,591.48,1002.508475


In [ ]:
part = '342A979721E'
part_list.append(part)
parts = tuple(part_list)

In [ ]:
part_src = parts_list[parts_list['part_num']=='342A979721D']
part_src = part_src.reset_index()
part_src

In [ ]:
# add info to sheet[1]
i_count = 0
po_name = 'FFVN-PO-110'
for i in range(16,1000):
    try:
        if sh.range(f'C{i}').value.startswith('FFVN'):
            info = sh.range(f'C{i}').value[6:-1]
        if sh.range(f'C{i}').value.startswith('Stock'):
            info = 'Stock'
#         print(info)
    except Exception as e:
        print (i,e)
#     print(i)
    if sh.range(f'D{i}').value != None: 
#         print(i,sh.range(f'D{i}').value.strip(),info.strip())
        part_number = sh.range(f'D{i}').value.strip()
        sh1.range('A5').offset(i_count).value = i_count + 1
        sh1.range('D5').offset(i_count).value = i_count + 1
        sh1.range('H5').offset(i_count).value = info
        sh1.range('I5').offset(i_count).value = sh.range(f'F{i}').value
        sh1.range('C5').offset(i_count).value = po_name
        try:
            part_src = parts_list[parts_list['part_num']==part_number]
            part_src = part_src.reset_index()




            sh1.range('E5').offset(i_count).value = part_src['SAP'][0]
            sh1.range('F5').offset(i_count).value = part_src['part_num'][0]
            sh1.range('G5').offset(i_count).value = part_src['PART_DESCRIPTION'][0]

            sh1.range('J5').offset(i_count).value = part_src['Sub Price'][0]
        except:
            sh1.range('F5').offset(i_count).value = part_number
            
        i_count +=1
    if sh.range(f'D{i}').value == 'Reviewed by':
        print('Done')
        break

In [ ]:
part_number

### Price for folder


In [13]:
import os
import xlwings as xw

In [19]:
mf = xw.Book()
sh = mf.sheets('Sheet1')

In [20]:
sh.range('A1').value = 'Model'
sh.range('B1').value = 'PartName'
sh.range('C1').value = 'PartNum'
sh.range('D1').value = 'Dealer Price'


In [21]:
i = 0
for file in os.listdir('inspection/HNNA'):
    print(file)
    org = xw.Book(f'inspection/HNNA/{file}')
    po = org.sheets('PartsOrder')
    model =''
    partname = ''
    partnum = ''
    for k in range(20):
        if po.range('F5').offset(k,0).value !='':
            model = po.range('H5').offset(k,0).value
            partname = po.range('G5').offset(k,0).value
            partnum = po.range('F5').offset(k,0).value
            
            #record to master
            sh.range('A2').offset(i,0).value = model
            sh.range('B2').offset(i,0).value = partname
            sh.range('C2').offset(i,0).value = partnum
            
            i +=1
        else:
            print(i)
            break
    
    
    org.close()

EB 530S.xls
EB 530T.xls
EC 530WL3 .xls
EG 530WR.xls
VP 3500.xls
XL 4450.xls


In [23]:
sh.autofit()